In [ ]:
!apt install python3-requests
!apt install python3-future
!apt install python3-certifi

In [ ]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 81kB 3.9MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.3-cp36-none-any.whl size=55590 sha256=296a87cf34ac93cae7f2f98aca9ed13dc46c1dbc6b6c4101554f9f31bd385d82
  Stored in directory: /root/.cache/pip/wheels/77/19/9c/cf58e8c83abec055f875508722503d29cdb6a34f4c58c7537a
Successfully built pyTelegramBotAPI


In [ ]:
%cd '/content/drive/My Drive/Reconocimiento De Emociones/data/'
%ls

/content/drive/My Drive/Reconocimiento De Emociones/data
202097164032rostrodetectado.jpg    202097204823rostrodetectado.jpg
202097165011rostrodetectado.jpg    202097204823rostrosindetectar.jpg
202097165011rostrosindetectar.jpg  202097204953rostrodetectado.jpg
202097165826rostrodetectado.jpg    202097204953rostrosindetectar.jpg
202097165826rostrosindetectar.jpg  20209720565rostrodetectado.jpg
202097171932rostrodetectado.jpg    20209720565rostrosindetectar.jpg
202097171932rostrosindetectar.jpg  202097211019rostrodetectado.jpg
202097171944rostrodetectado.jpg    202097211019rostrosindetectar.jpg
202097171944rostrosindetectar.jpg  202097211112rostrodetectado.jpg
202097203235rostrodetectado.jpg    202097211112rostrosindetectar.jpg
202097203235rostrosindetectar.jpg  202097211149rostrodetectado.jpg
202097203316rostrodetectado.jpg    202097211149rostrosindetectar.jpg
202097203316rostrosindetectar.jpg  202097211226rostrodetectado.jpg
202097203520rostrodetectado.jpg    202097211226rostrosindetect

In [ ]:
import telebot
import requests
from base64 import b64decode
import os
from __future__ import print_function
import requests
import json
import cv2
import time
from google.colab.patches import cv2_imshow

TOKEN = '1137373995:AAFADmzWmRqMZn19dPqARXu81rvftBxGVnU' # Ponemos nuestro Token generado con el @BotFather
bot  = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, Bienvenido enviame una imagen para detectar tu emocion :)")

@bot.message_handler(commands=['help'])
def send_welcome(message):
  bot.send_message(message.chat.id,"Necesitas Ayuda")
 
@bot.message_handler(content_types=['photo'])
def handle_docs_audio(message):
	  enviarmensaje(message)
   
@bot.message_handler(func=lambda message: True)
def echo_all(message):
	bot.reply_to(message, message.text)
  

def enviarmensaje(message):
    
    try:
      #obtener las imagenes de mi telegram
      print(message.photo[2].file_id)
      file_info = bot.get_file(message.photo[2].file_id)
      file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(TOKEN, file_info.file_path))
      # fin de obtener la imagen

      # guardar mi imagen de telegram
      raw = message.photo[2].file_id
      path = "rostro.jpg"
      file_info = bot.get_file(raw)
      downloaded_file = bot.download_file(file_info.file_path)
      with open(path,'wb') as new_file:
          new_file.write(downloaded_file)
      # fin de guardar imagen

      #consultar a la api para mi reconocimiento de emociones
      addr = 'http://f5fd960d93a7.ngrok.io'
      test_url = addr + '/emociones'
      content_type = 'image/jpeg'
      headers = {'content-type': content_type}
      img = cv2.imread(path)
      _,img_encoded = cv2.imencode('.jpg', img)
      response = requests.post(test_url, data=img_encoded.tostring(), headers=headers)
      r = json.loads(response.text)
      # fin de consultar a la api

      # Enviando mensajes e imagen de espera ..
      bot.send_message(message.chat.id, "He recibido tu imagen")
      bot.send_message(message.chat.id, "procesando imagen")
      doc = open('cargando-loading-045.gif', 'rb')
      bot.send_document(message.chat.id, doc)
      # FIn de enviando mensajes

      # Respuesta de la api
      mensaje_respuesta = 'Tu emocion es : ' + r["emocion"]
      print(mensaje_respuesta)
      print(r["nombre_imagen"])
      rutaimg = r["nombre_imagen"]
      # Fin de la respuesta

      # tiempo muerto porque no se actualiza la imagen guardad en gdrive
      time.sleep(30)
      # enviando mensaje
      bot.reply_to(message, "Gracias por Esperar")
      bot.reply_to(message, mensaje_respuesta)
      print("mensaje luego de la espera")
      ruta_emo = emociones(r["emocion"])
      print(ruta_emo)
      photo = open(rutaimg, 'rb')
      #bot.send_photo('1189691411', photo)
      bot.send_photo(message.chat.id, photo)
      photo2 = open(ruta_emo, 'rb')
      bot.send_photo(message.chat.id, photo2)
      # fin
    except:
      print("Ups A ocurrido un error vuelve a intentarlo")
      bot.send_message(message.chat.id, "Ups A ocurrido un error vuelve a intentarlo")
      photo3 = open('fallastecnicas.jpg', 'rb')
      bot.send_photo(message.chat.id, photo3)

def emociones (emocion):
    ruta = '/content/drive/My Drive/Reconocimiento De Emociones/src/Emojis/'
    if emocion == 'Enojado':
        return ruta + 'Enojado.jpg'
    elif emocion == 'Disgustado':
         return ruta + 'Disgustado.jpg'
    elif emocion == 'Temerosa':
         return ruta + 'asustado.jpg'
    elif emocion == 'Feliz':
         return ruta + 'Feliz.jpg'
    elif emocion == 'Neutral':
         return ruta + 'Neutral.jpg'
    elif emocion == 'Triste':
         return ruta + 'tristeza.jpeg'
    elif emocion == 'Sorprendido':
         return ruta + 'sorprendido.jpg'
    else:
         return ruta + 'Neutral.jpg'

bot.polling()

In [ ]:
#raw = message.photo[2].file_id
#path = raw+".jpg"
#file_info = bot.get_file(raw)
#downloaded_file = bot.download_file(file_info.file_path)
#with open(path,'wb') as new_file:
#    new_file.write(downloaded_file)

# enviar imagen
#photo = open('/content/lena.jpg', 'rb')
#bot.send_photo('1189691411', photo)